In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Layout, widgets
from IPython.display import display, clear_output
import requests
import time
from scipy.signal import kaiser_atten, freqz, convolve, welch
import warnings
warnings.filterwarnings('ignore')

# Define your input widgets for the frequencies
f1_input = widgets.IntText(value=600, description='f1 (Hz):', continuous_update=False)
f2_input = widgets.IntText(value=1100, description='f2 (Hz):', continuous_update=False)

# Define the output area for the plots
plot_output = widgets.Output()

def process_signal(change):
    f1, f2 = f1_input.value, f2_input.value
    
    if any(freq < 0 for freq in [f1, f2]) or any(freq > 4000 for freq in [f1, f2]):
        # Clear the output and display a warning message if any frequency is greater than 4000
        with plot_output:
            plot_output.clear_output(wait=True)
            warning_html = widgets.HTML(
                value="<div style='color: black; background-color: #ffffcc; padding: 10px; border-radius: 5px; font-size: 16px; text-align: center;'>" 
                      "<b>⚠️ Warning:</b> Frequency should not exceed 4000 and must be non-negative!</div>",
                placeholder='',
                description='',
            )
            display(warning_html)
    else:
        # Proceed with the plot if all frequencies are within the limit
        with plot_output:
            plot_output.clear_output(wait=True)
            plt.close('all')  # Close all existing figures to prevent memory leaks

            # Replace the URL with the URL of the raw content
            url = "https://raw.githubusercontent.com/ntua-el17840/Interactive-Digital-Communications/main/test-book/_static/sima.txt"
            response = requests.get(url)

            # Split the response text by new lines and convert each line to a float
            s = np.array([float(line) for line in response.text.splitlines()])
            
            Fs = 8192

            f2m1 = f2 - f1
            f2p1 = (f2 + f1) / 2
            N = 256
            Ts = 1 / Fs
            t = np.arange(-(N - 1), N, 2) * Ts / 2

            # Bandpass filter
            hbp = 2 / Fs * np.cos(2 * np.pi * f2p1 * t) * np.sinc(f2m1 * t)
            hbpw = hbp * kaiser_atten(len(hbp), 5)

            # Compute frequency responses
            w, h = freqz(hbp, worN=8000)
            w_w, h_w = freqz(hbpw, worN=8000)
            fs = Fs  # Sampling frequency for normalization

            # Creating a figure and a 1x2 subplot layout
            fig, axs = plt.subplots(1, 2, figsize=(14, 4))

            # Plotting the impulse responses on the first subplot
            axs[0].plot(hbp, label='Bandpass Filter', color='b')
            axs[0].plot(hbpw, label='Windowed Bandpass Filter', color='r')
            axs[0].set_title('Impulse Responses')
            axs[0].set_xlabel('Sample')
            axs[0].set_ylabel('Amplitude')
            axs[0].legend()
            axs[0].grid(True)

            # Plotting the frequency responses on the second subplot
            axs[1].plot(w / np.pi * (fs / 2), 20 * np.log10(abs(h)), label='Bandpass Filter', color='b')
            axs[1].plot(w_w / np.pi * (fs / 2), 20 * np.log10(abs(h_w)), label='Windowed Bandpass Filter', color='r')
            axs[1].set_title('Frequency Responses')
            axs[1].set_xlabel('Frequency [Hz]')
            axs[1].set_ylabel('Magnitude [dB]')
            axs[1].legend()
            axs[1].grid(True)

            # Convolution with the signal
            sima_bp = convolve(s, hbp, mode='same')
            sima_bpw = convolve(s, hbpw, mode='same')

            fig, axs = plt.subplots(1, 2, figsize=(14, 4))

            # Plotting the Power Spectral Density
            f, Pxx = welch(sima_bp, Fs, nperseg=1024)
            axs[0].semilogy(f, Pxx)
            axs[0].set_title('Power Spectral Density of Bandpass Filtered Signal')
            axs[0].set_xlabel('Frequency [Hz]')
            axs[0].set_ylabel('PSD [V**2/Hz]')
            axs[0].grid(True)

            f, Pxx = welch(sima_bpw, Fs, nperseg=1024)
            axs[1].semilogy(f, Pxx)
            axs[1].set_title('Power Spectral Density of Windowed Bandpass Filtered Signal')
            axs[1].set_xlabel('Frequency [Hz]')
            axs[1].set_ylabel('PSD [V**2/Hz]')
            axs[1].grid(True)

            plt.tight_layout()
            plt.show()

# Attach the event handler to the frequency inputs
f1_input.observe(process_signal, names='value')
f2_input.observe(process_signal, names='value')

# Display widgets
input_ui = widgets.VBox([f1_input, f2_input])
ui = widgets.HBox([input_ui], layout=Layout(align_items='center'))

display(ui, plot_output)

# Call process_signal initially
process_signal(None)


Output()